<a href="https://colab.research.google.com/github/wael-slimi/bankSystem/blob/main/Speciality_Recommedation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'doctor-specialist-recommendation-system:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3417941%2F5969573%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240425%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240425T013615Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8dda38a1aa51ec94e6775182a6fb7006fc3c258741f19b3788113b0575017ab8b8d98e740f41be01b962b7dc4a55304fc11140ad308960606a32ba16c71e429ea27a8a520f377828ec66a11a4743c933928733b99caaf31596f39137f23e74a2a5af7106d9dccd808de3ece05374424c63db4fbf0487a9612ce8d9438621e71d530304ea24e7a33e0ac9b7b8a8287ac9d4857b0edd919a4856dcfaae6bec3824272c35cdeacc3fa0930102fba82cf9d812aa5e8010b16d833e9288b60cb1549ad3191792dfeadd37fe2cd2364ad694f8c731655e16ce692df53b3717799d20cc42a96f16560b99b0d8281019fe597988dc69c929dd4c9be558a7f728b4cadb76'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 942184 bytes downloaded
Downloaded and uncompressed: doctor-specialist-recommendation-system
Data source import complete.


### Predicting Disease based on the Symptoms and identifying specialist based on predicted disease

**The dataset contains information about symptoms, diseases, recommended doctors for each disease, and descriptions of the diseases. Using a classification model, diseases have been predicted based on the given symptoms. The predictions provide insights into the predicted disease, the likelihood or chance of that disease based on the classification model, the recommended doctor to visit for that disease, and a description of the predicted disease.**

**The summary output includes:**

**Predicted Disease:** *-The disease that has been predicted based on the symptoms provided.-*

**Chance of the Disease:** *-The likelihood or probability of having that disease, as determined by the classification model.-*

**Recommended Doctor:** *-The doctor or specialist recommended to visit for further evaluation or treatment of the predicted disease.-*

**Description of the Disease:** *-A brief overview or description of the predicted disease, providing additional information about its symptoms, causes, and potential treatments.-*

**This summary aims to provide a concise and informative overview of the predicted disease, its associated likelihood, the recommended doctor, and a brief description to aid in understanding and decision-making regarding the predicted disease.**

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from collections import Counter

In [7]:
pd.set_option('display.max_colwidth', -1)

ValueError: Must provide an even number of non-keyword arguments

In [6]:
pd.set_option('display.max_colwidth', 50)

In [8]:
dis_sym_data = pd.read_csv("/kaggle/input/doctor-specialist-recommendation-system/Original_Dataset.csv")

In [12]:
dis_sym_data.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
dis_sym_data.shape

(4920, 18)

**Finding unique values across all the symptoms column**

In [14]:
columns_to_check = []
for col in dis_sym_data.columns:
    if col != 'Disease':
        columns_to_check.append(col)

In [15]:
symptoms = dis_sym_data.iloc[:, 1:].values.flatten()
symptoms = list(set(symptoms))

**Convert Symptoms to Binary Columns**

In [16]:
for symptom in symptoms:
    dis_sym_data[symptom] = dis_sym_data.iloc[:, 1:].apply(lambda row: int(symptom in row.values), axis=1)

dis_sym_data_v1 = dis_sym_data.drop(columns=columns_to_check)

In [17]:
dis_sym_data_v1 = dis_sym_data_v1.loc[:, dis_sym_data_v1.columns.notna()]

In [18]:
dis_sym_data_v1.shape

(4920, 132)

In [19]:
dis_sym_data_v1.columns = dis_sym_data_v1.columns.str.strip()

In [20]:
dis_sym_data_v1.columns

Index(['Disease', 'distention_of_abdomen', 'knee_pain', 'bladder_discomfort',
       'constipation', 'nodal_skin_eruptions', 'mucoid_sputum', 'swollen_legs',
       'continuous_feel_of_urine', 'excessive_hunger',
       ...
       'shivering', 'sunken_eyes', 'swelling_of_stomach',
       'abnormal_menstruation', 'abdominal_pain', 'mood_swings',
       'foul_smell_of urine', 'chest_pain', 'chills', 'passage_of_gases'],
      dtype='object', length=132)

**Encoding Disease**

In [21]:
var_mod = ['Disease']
le = LabelEncoder()
for i in var_mod:
    dis_sym_data_v1[i] = le.fit_transform(dis_sym_data_v1[i])

In [22]:
X = dis_sym_data_v1.drop(columns="Disease")
y = dis_sym_data_v1['Disease']

**Model Generation**

In [23]:
def class_algo(model,independent,dependent):
    model.fit(independent,dependent)
    pred = model.predict(independent)
    accuracy = metrics.accuracy_score(pred,dependent)
    print(model_name,'Accuracy : %s' % '{0:.3%}'.format(accuracy))

In [24]:
algorithms = {'Logistic Regression':
              {"model": LogisticRegression()},

              'Decision Tree':
              {"model": tree.DecisionTreeClassifier()},

              'Random Forest':
              {"model": RandomForestClassifier()},

              'SVM':
              {"model": svm.SVC(probability=True)},

              'NaiveBayes' :
              {"model": GaussianNB()},

              'K-Nearest Neighbors' :
              {"model": KNeighborsClassifier()},
             }

In [25]:
for model_name, values in algorithms.items():
    class_algo(values["model"],X,y)

Logistic Regression Accuracy : 100.000%
Decision Tree Accuracy : 100.000%
Random Forest Accuracy : 100.000%
SVM Accuracy : 100.000%
NaiveBayes Accuracy : 100.000%
K-Nearest Neighbors Accuracy : 100.000%


**Map Description and Specialist for the Disease Predicted**

In [26]:
doc_data = pd.read_csv("/kaggle/input/doctor-specialist-recommendation-system/Doctor_Versus_Disease.csv",encoding='latin1', names=['Disease','Specialist'])

In [27]:
doc_data.tail(5)

,Disease,Specialist
36,Bronchial Asthma,Pulmonologist
37,Pneumonia,Pulmonologist
38,Osteoarthristis,Rheumatologists
39,Arthritis,Rheumatologists
40,Tuberculosis,Tuberculosis


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('Disease').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('Specialist').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Specialist'].value_counts()
    for x_label, grp in _df_2.groupby('Disease')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Disease')
_ = plt.ylabel('Specialist')

 -*Doctor name is wrong for Disease - Tuberculosis. Replace to Pulmonologist*-

In [28]:
doc_data['Specialist'] = np.where((doc_data['Disease'] == 'Tuberculosis'),'Pulmonologist', doc_data['Specialist'])

In [29]:
doc_data.tail(5)

,Disease,Specialist
36,Bronchial Asthma,Pulmonologist
37,Pneumonia,Pulmonologist
38,Osteoarthristis,Rheumatologists
39,Arthritis,Rheumatologists
40,Tuberculosis,Pulmonologist


In [30]:
des_data = pd.read_csv("/kaggle/input/doctor-specialist-recommendation-system/Disease_Description.csv")

In [31]:
des_data.head()

,Disease,Description
0,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...
1,Malaria,An infectious disease caused by protozoan para...
2,Allergy,An allergy is an immune system response to a f...
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroi..."
4,Psoriasis,Psoriasis is a common skin disorder that forms...


**Test with Unknown Data**

In [32]:
test_col = []
for col in dis_sym_data_v1.columns:
    if col != 'Disease':
        test_col.append(col)


test_data = {}
symptoms = []
predicted = []
def test_input():
    symptoms.clear()
    predicted.clear()
    num_inputs = int(input("Enter the number of symptoms you have: "))
    for i in range(num_inputs):
        user_input = input("Enter Symptoms #{}: ".format(i+1))
        symptoms.append(user_input)
    print("Symptoms you have:", symptoms)
    for column in test_col:
        test_data[column] = 1 if column in symptoms else 0
        test_df = pd.DataFrame(test_data, index=[0])
    print("Predicting Disease based on 6 ML algorithms...")
    for model_name, values in algorithms.items():
        predict_disease = values["model"].predict(test_df)
        predict_disease = le.inverse_transform(predict_disease)
        predicted.extend(predict_disease)
    disease_counts = Counter(predicted)
    percentage_per_disease = {disease: (count / 6) * 100 for disease, count in disease_counts.items()}
    result_df = pd.DataFrame({"Disease": list(percentage_per_disease.keys()),
                               "Chances": list(percentage_per_disease.values())})
    result_df = result_df.merge(doc_data, on='Disease', how='left')
    result_df = result_df.merge(des_data, on='Disease', how='left')
    return result_df

In [33]:
test_input()

Enter the number of symptoms you have: 6
Enter Symptoms #1: itching	
Enter Symptoms #2: 
Enter Symptoms #3: 
Enter Symptoms #4: 
Enter Symptoms #5: 
Enter Symptoms #6: 
Symptoms you have: ['itching\t', '', '', '', '', '']
Predicting Disease based on 6 ML algorithms...


,Disease,Chances,Specialist,Description
0,Allergy,33.333333,Allergist,An allergy is an immune system response to a f...
1,AIDS,50.000000,Osteopathic,Acquired immunodeficiency syndrome (AIDS) is a...
2,Arthritis,16.666667,Rheumatologists,Arthritis is the swelling and tenderness of on...
